In [1]:
using Flux
using Flux.Tracker
using CSV, DataFrames
using Missings

In [2]:
# Processing the data
f = CSV.read("./PressureData/Stations_2018.csv")

Nw = 25                                      # Window size
Nc = length(f)-1                             # Number of columns
T = Int64(floor(length(f[:,1]) / Nw) * Nw)   # Number of data points


s = Array{Float64, 2}(Nc*Nw,Int64(T/Nw))

# count number of missing values
num = 0
for (name,col) in eachcol(f[:,2:Nc+1])
    for i=1:length(col)
        if ismissing(col[i])
            num += 1
        end
    end
end
println("number of missing elements: $num")

# linear interpolation to estimate missing values
for (name,col) in eachcol(f[:,2:Nc+1])
    count = 0
    for i=1:length(col)
        if ismissing(col[i])
            count += 1
        end
        if count != 0 && !ismissing(col[i])
            dif = (col[i]-col[i-count-1]) / (count + 1)
            for j = 1:count
                col[i-count-1+j] = col[i-count-2+j] + dif
            end
            count = 0
        end
    end
end

# count number of missing values
num = 0
for (name,col) in eachcol(f[:,2:Nc+1])
    for i=1:length(col)
        if ismissing(col[i])
            num += 1
        end
    end
end
println("number of missing elements: $num")

# convert raw data to input format
for i = 1:Int64(T/Nw)
    s[:,i] .= vec(convert(Array, f[Nw*(i-1)+1:Nw*i,2:Nc+1]))
end

number of missing elements: 2332
number of missing elements: 0


In [3]:
function create(s, Nc, Nw, T, λ)
    # common sub-expressions
    Ncw = Nc * Nw
    df1 = Int64(length(s[1,:]) / 2)
    df2 = Int64(df1 / 97)
    
    # encoder weights saved to use in decoder weight creation
    W1a = rand(df1,Ncw)
    W2a = rand(df2, df1)
        
    W1 = param(W1a)
    b1 = param(rand(df1))
    layer1(x) = W1 * x .+ b1

    W2 = param(W2a)
    b2 = param(rand(df2))
    layer2(x) = W2 * x .+ b2

    W3 = param(W2a')
    b3 = param(rand(df1))
    layer3(x) = W3 * x .+ b3

    W4 = param(W1a')
    b4 = param(rand(Ncw))
    layer4(x) = W4 * x .+ b4

    m(x) = relu.(layer4(relu.(layer3(relu.(layer2(relu.(layer1(x))))))))
    loss(x) = Flux.mse(x,m(x))
    
    l = 0
    for i = 1:Int64(T/Nw)
        l += loss(s[:,i])
    end
    
    l = loss(s[:,1])
    l += λ *(reduce(+, W1 .^ 2) + reduce(+, W2 .^ 2))

    println(l)
    
    back!(l)

end

create (generic function with 1 method)

In [4]:
function update(model, α)
    model.layer1.W1.data -= α * (model.layer1.W1.grad + model.layer1.W4.grad')
    model.layer1.W4.data -= α * (model.layer1.W1.grad' + model.layer1.W4.grad)
    
    model.layer1.W2.data -= α * (model.layer1.W2.grad + model.layer1.W3.grad')
    model.layer1.W3.data -= α * (model.layer1.W2.grad' + model.layer1.W3.grad)
    
    model.layer1.b1.data -= α * model.layer1.b1.grad
    model.layer1.b2.data -= α * model.layer1.b2.grad
    model.layer1.b3.data -= α * model.layer1.b3.grad
    model.layer1.b4.data -= α * model.layer1.b4.grad
end

update (generic function with 1 method)

In [ ]:
model = create(s, Nc, Nw, T, 0.1)

for i=1:1e4
    update(model, 0.1)
end

println(mse(s[:,1],model(s[:,1])))